In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
BASE_DIR = Path("model.ipynb").resolve().parent.parent
csv_path = BASE_DIR / "converted-data" / "enrolments.csv"

enrolments = pd.read_csv(csv_path)
enrolments

,student,exam
0,A890186790,R13001E1
1,A890186790,R13006E1
2,A890186790,R13016E1
3,A890186790,R13021E1
4,A890186790,R13022E1
...,...,...
33992,F947812713,J51IMAE1
33993,F948091126,H31BDAE1
33994,F948091126,H31DM1E1
33995,F948091126,HGAEM2E1


Setup:

`students_exams`: Dictionary with keys are students and values are required exams.
`exams` : List of exams
`timeslots` : Timeslots for each exams to take place

In [3]:
students_exams = enrolments.groupby("student")["exam"].apply(list).to_dict()
exams = enrolments["exam"].unique().tolist()
timeslots = range(1, 32)

Model 1: Constraints:
- Each exam scheduled exactly once
- No student has overlapping exams

In [85]:
m = gp.Model("ExamScheduling1")

# Decision variables: x[e,t] = 1 if exam e scheduled at timeslot t
x = {}
for e in exams:
    for t in timeslots:
        x[e,t] = m.addVar(vtype=GRB.BINARY, name=f"x_{e}_{t}")

# Constraint 1: Each exam scheduled exactly once
for e in exams:
    m.addConstr(sum(x[e,t] for t in timeslots) == 1, name=f"one_slot_{e}")

# Constraint 2: No student has overlapping exams
for s, s_exams in students_exams.items():
    for t in timeslots:
        for i in range(len(s_exams)):
            for j in range(i+1, len(s_exams)):
                e1 = s_exams[i]
                e2 = s_exams[j]
                m.addConstr(x[e1,t] + x[e2,t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}")

# Objective: just find feasible schedule
m.setObjective(0, GRB.MINIMIZE)

# Optimize
m.optimize()

# Print schedule
print("Exam schedule:")
for e in exams:
    for t in timeslots:
        if x[e,t].X > 0.5:
            print(f"Exam {e} scheduled at timeslot {t}")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1986443 rows, 24800 columns and 3996086 nonzeros
Model fingerprint: 0xeeb25c8e
Variable types: 0 continuous, 24800 integer (24800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.17 seconds (0.19 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Exam schedule:
Exam R13001E1 scheduled at timeslot 24
Exam R13006E1 scheduled at timeslot 1
Exam R13016E1 scheduled at timeslot 19
Exam R13021E1 scheduled at timeslot 11
Exam R13022E1 s

In [127]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.layouts import column
output_notebook()

# Settings
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
slots_per_day = {"Mon":3, "Tue":3, "Wed":3, "Thu":3, "Fri":3, "Sat":1}
total_weeks = 2

# Build coordinates for each timeslot
x_coords = []
y_coords = []
exam_texts = []

slot_counter = 0
for week in range(total_weeks):
    y_offset = week  # offset for second week to separate visually
    for day_idx, day in enumerate(days):
        for slot_num in range(1, slots_per_day[day]+1):
            x_coords.append(day)
            y_coords.append(slot_num + y_offset)
            # Assign exams sequentially for demo; replace with your schedule
            exams_in_slot = []
            if slot_counter < len(exams):
                exams_in_slot.append(exams[slot_counter])
            exam_texts.append(", ".join(exams_in_slot))
            slot_counter += 1

# Create ColumnDataSource
source = ColumnDataSource(data=dict(x=x_coords, y=y_coords, exams=exam_texts))

# Figure
p = figure(
    x_range=days,
    y_range=(-3, max(y_coords)+1),
    title="Two-week Exam Schedule",
    tools="tap"
)

# Draw rectangles for timeslots
p.rect(x='x', y='y', width=0.9, height=0.8, source=source, fill_color="skyblue", line_color="black")

# Callback: show exams in clicked timeslot
callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const selected = source.selected.indices;
    if(selected.length > 0){
        const exams = data['exams'][selected[0]];
        const slot = data['x'][selected[0]] + ", slot " + data['y'][selected[0]];
        alert('Timeslot: ' + slot + '\\nExams:\\n' + exams);
    }
""")
source.selected.js_on_change('indices', callback)

# Style
p.xaxis.axis_label = "Day of Week"
p.yaxis.axis_label = "Timeslot"
p.xaxis.major_label_orientation = 1.2

show(p)


Loading BokehJS ...